# Publishers - Sending Data to the Create3 

In ROS2, we get send data to a Robot by **publishing** to a topic. For this topic let's get the battery information from the Create3.

To do this from the command-line, we use the command `ros2 topic pub <topic_name> <msg_type> '<args>'`. 

The `<topic_name>` argument is the topic we want to publish to. This time around there are two additional arguments though `<msg_type>` and `'<args>'`. 

The `<msg_type>` argument is the form the message you are sending will take.

The `'<args>'` argument is the actual data you’ll pass to the topic. To find the structure of the arguments check out the last section of this notebook.

In this case let's change the color of the LED's on our Create3, to do this we will set up the arguments as follows.

```
<topic_name> = /cmd_lightring
<msg_type>   = irobot_create_msgs/msg/LightringLeds
'<args>'     = "{override_system: true, leds: [{red: 255, green: 0, blue: 0}, {red: 0, green: 255, blue: 0}, {red: 0, green: 0, blue: 255}, {red: 255, green: 255, blue: 0}, {red: 255, green: 0, blue: 255}, {red: 0, green: 255, blue: 255}]}"
```

Note: Typically the `ros2 topic pub` command is called from the command-line of a terminal, here we use the `%%bash` magic word to simulate the command line in this jupyter notebook. `--once` is an optional argument meaning “publish one message then exit”.


In [ ]:
%%bash 

ros2 topic pub --once /cmd_lightring irobot_create_msgs/msg/LightringLeds "{override_system: true, leds: [{red: 255, green: 0, blue: 0}, {red: 0, green: 255, blue: 0}, {red: 0, green: 0, blue: 255}, {red: 255, green: 255, blue: 0}, {red: 255, green: 0, blue: 255}, {red: 0, green: 255, blue: 255}]}"

Publishing this message should change all of the colors on your Create3 to Blue.

In the command-line this isn't much help to us, so next let's see how to get send information in a python file.

## LED Publisher Example Code

In [ ]:
import sys
import rclpy
from rclpy.node import Node

from irobot_create_msgs.msg import LedColor
from irobot_create_msgs.msg import LightringLeds


class ColorPalette():
    '''
    Helper Class to define frequently used colors
    '''
    def __init__(self):
        self.red = LedColor(red=255, green=0, blue=0)
        self.green = LedColor(red=0, green=255, blue=0)
        self.blue = LedColor(red=0, green=0, blue=255)
        self.yellow = LedColor(red=255, green=255, blue=0)
        self.pink = LedColor(red=255, green=0, blue=255)
        self.cyan = LedColor(red=0, green=255, blue=255)
        self.purple = LedColor(red=127, green=0, blue=255)
        self.white = LedColor(red=255, green=255, blue=255)
        self.grey = LedColor(red=189, green=189, blue=189)
        self.tufts_blue = LedColor(red=98, green=166, blue=10)
        self.tufts_brown = LedColor(red=94, green=75, blue=60)


class LEDPublisher(Node):
    '''
    Publish a custom color to each LED on the Create3
    '''

    def __init__(self, namespace: str = ""):
        super().__init__('led_publisher')

        self.cp = ColorPalette()
        self.lights_publisher = self.create_publisher(
            LightringLeds, namespace + '/cmd_lightring', 10)
    
        timer_period = 0.5  # seconds
        self.timer = self.create_timer(timer_period, self.timer_callback)
        self.lightring = LightringLeds()
        self.lightring.override_system = True

    def timer_callback(self):
        '''
        Purpose
        -------
        This function will be called every 0.5 seconds, right now it's 
        publishing the same colors every time. 
        Try and see if you can get the colors to change each time this function 
        is called!
        There are six LEDs, so we have a list of six colors
        '''
        # Define the variables we want to send to the topic
        led_colors = [self.cp.purple, self.cp.cyan, self.cp.pink, 
                      self.cp.blue, self.cp.red, self.cp.green] # EDIT THIS LINE TO CHANGE THE COLOR
        current_time = self.get_clock().now()

        # Pack them into one message using format defined for irobot_create_msgs/msg/LightringLeds
        self.lightring.leds = led_colors
        self.lightring.header.stamp = current_time.to_msg()
        
        self.lights_publisher.publish(self.lightring) # Publish!

    def reset(self):
        '''
        Purpose
        -------
        Release control of the LEDs back to the Create3.
        '''
        white = [self.cp.white, self.cp.white, self.cp.white,
                 self.cp.white, self.cp.white, self.cp.white]
        current_time = self.get_clock().now()
        
        self.lightring.override_system = False
        self.lightring.leds = white
        self.lights_publisher.publish(self.lightring)

def main(args=None):
    rclpy.init(args=args)

    led_publisher = LEDPublisher()
    try:
        rclpy.spin(led_publisher)
    except KeyboardInterrupt:
        print('\nCaught Keyboard Interrupt')
    except BaseException:
        print('Exception:', file=sys.stderr)
    finally:
        print("Done")
        # Destroy the node explicitly
        # (optional - otherwise it will be done automatically
        # when the garbage collector destroys the node object)
        led_publisher.reset()
        led_publisher.destroy_node()
        rclpy.shutdown()


if __name__ == '__main__':
    main()